<a href="https://colab.research.google.com/github/shouvikcirca/LLMs/blob/main/Audio_RAG_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers==4.37.2
! pip install torch
! pip install gradio
! pip install -U datasets
! pip install librosa
! pip install openai
! pip install python-dotenv
! pip install google-generativeai
! pip install chromadb
! pip install langchain-google-genai
! pip install langchain
! pip install langchain-community

In [2]:
from transformers import pipeline
import torch
import gradio as gr
import numpy as np
import librosa
import openai
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from langchain_community.document_loaders.telegram import text_to_docs
from langchain.chains.combine_documents import create_stuff_documents_chain
import google.generativeai as genai

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [16]:
!ls drive/MyDrive/LLM/chroma_db

4562bfcd-74b5-4703-b8e2-0030a0868561  855bd276-096b-4bad-8022-45c39a4877fa  chroma.sqlite3


In [9]:
_= load_dotenv('drive/MyDrive/env')
openai_api_key = os.environ['OPENAI_API_KEY']
gemini_api_key = os.environ['GEMINI_API_KEY']

audioClient = openai.OpenAI(api_key = openai_api_key)
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=gemini_api_key)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=gemini_api_key)

In [20]:
vectordb = Chroma(
    persist_directory='drive/MyDrive/LLM/chroma_db/',
    embedding_function=embeddings
    collection_name=''
)

388

In [21]:
vectordb.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [23]:
demo = gr.Blocks()

In [ ]:
asr = pipeline(task="automatic-speech-recognition",model="distil-whisper/distil-small.en")

In [ ]:
def transcribe_speech(audioSample):
  audioSampleReshaped = audioSample[1].astype(float)
  audioResampled = librosa.resample(audioSampleReshaped, orig_sr=44100,target_sr=16000)
  output = asr(audioResampled)
  return output["text"]

In [ ]:
mic_transcribe = gr.Interface(
    fn = transcribe_speech,
    inputs = gr.Audio(sources="microphone",type='numpy'),
    outputs = gr.Textbox(label="Transcription", lines=3),
    allow_flagging='never'
)

In [ ]:
mic_transcribe.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://ca55ff2210e2f5ed4e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
mic_transcribe.close()

Closing server running on port: 7860


In [ ]:
#For checking sampling rate
# import numpy as np
# at = np.transpose(audioSample)
# import librosa
# audio_mono = librosa.to_mono(at)\
# aresampled = librosa.resample(audio_mono, orig_sr=16000,target_sr=16000)
asr.feature_extractor.sampling_rate

In [ ]:
translator = pipeline(task="translation", model="facebook/nllb-200-distilled-600M", torch_dtype=torch.bfloat16)

In [ ]:
text_translated

[{'translation_text': 'How are you?'}]